In [1]:
from gklearn.utils.graphfiles import loadDataset
import networkx as nx
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from __future__ import print_function
import torch

In [3]:
def label_to_color(label):
    if label == 'C':
        return 0.1
    elif label == 'O':
        return 0.8
    
def nodes_to_color_sequence(G):
    return [label_to_color(c[1]['label'][0]) for c in G.nodes(data=True)]

Gs,y = loadDataset('/home/luc/TRAVAIL/DeepGED/MAO/dataset.ds')
#for e in Gs[13].edges():
#    print(Gs[13][e[0]][e[1]]['bond_type'])
print('edge max label',max(max([[G[e[0]][e[1]]['bond_type'] for e in G.edges()] for G in Gs])))
G1 = Gs[1]
G2 = Gs[9]
print(y[1],y[9])
plt.figure(0)
nx.draw_networkx(G1,with_labels=True,node_color = nodes_to_color_sequence(G1),cmap='autumn')

plt.figure(1)
nx.draw_networkx(G2,with_labels=True,node_color = nodes_to_color_sequence(G2),cmap='autumn')

plt.show()
import extended_label
for g in Gs:
    extended_label.compute_extended_labels(g)
#for v in Gs[10].nodes():
#        print(Gs[10].nodes[v])

#print(nx.to_dict_of_lists(Gs[13]))



#dict={'C':0,'N':1,'O':2}
#A,labels=from_networkx_to_tensor2(Gs[13],dict)
#print(A)
#A1=(A==torch.ones(13,13)).int()
#A2=(A==2*torch.ones(13,13)).int()
#print(A1)
#print(A2)


FileNotFoundError: [Errno 2] No such file or directory: '/home/luc/TRAVAIL/DeepGED/MAO/dataset.ds'

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import svd
from svd import iterated_power as compute_major_axis

#torch.autograd.set_detect_anomaly(True)

class Net(nn.Module):
    
        
    def __init__(self,GraphList,normalize=False,node_label='label'):
        super(Net, self).__init__()   
        self.normalize=normalize
        self.node_label=node_label
        dict,self.nb_edge_labels=self.build_node_dictionnary(GraphList)
        self.nb_labels=len(dict)
        print(self.nb_edge_labels)
        self.device=torch.device("cuda:0")
        nb_node_pair_label=self.nb_labels*(self.nb_labels-1)/2.0
        nb_edge_pair_label=int(self.nb_edge_labels*(self.nb_edge_labels-1)/2)
        
        self.node_weighs=nn.Parameter(torch.tensor(1.0/(nb_node_pair_label+nb_edge_pair_label+2))+(1e-3)*torch.rand(int(self.nb_labels*(self.nb_labels-1)/2+1),requires_grad=True,device=self.device)) # all substitution costs+ nodeIns/Del. old version: 0 node subs, 1 nodeIns/Del, 2 : edgeSubs, 3 edgeIns/Del        
        self.edge_weighs=nn.Parameter(torch.tensor(1.0/(nb_node_pair_label+nb_edge_pair_label+2))+(1e-3)*torch.rand(nb_edge_pair_label+1,requires_grad=True,device=self.device)) #edgeIns/Del
        
        self.card=torch.tensor([G.order() for G in GraphList]).to(self.device)
        card_max=self.card.max()
        self.A=torch.empty((len(GraphList),card_max*card_max),dtype=torch.int,device=self.device)
        self.labels=torch.empty((len(GraphList),card_max),dtype=torch.int,device=self.device)
        print(self.A.shape)
        for k in range(len(GraphList)):
            A,l=self.from_networkx_to_tensor(GraphList[k],dict)             
            self.A[k,0:A.shape[1]]=A[0]
            self.labels[k,0:l.shape[0]]=l
        print('adjacency matrices',self.A)
        print('node labels',self.labels)
        print('order of the graphs',self.card)
        
    def forward(self, input):        
        ged=torch.zeros(len(input)).to(self.device) 
        node_costs,nodeInsDel,edge_costs,edgeInsDel=self.from_weighs_to_costs()
        
            
        
        #print('weighs:',self.weighs.device,'device:',self.device,'card:',self.card.device,'A:',self.A.device,'labels:',self.labels.device)
        for k in range(len(input)):            
            g1=input[k][0]
            g2=input[k][1]
            n=self.card[g1]
            m=self.card[g2]
            C=self.construct_cost_matrix(g1,g2,node_costs,edge_costs,nodeInsDel,edgeInsDel)      
            #S=self.mapping_from_similarity(C,n,m)
            S=self.mapping_from_cost(C,n,m)            
            v=torch.flatten(S)
            
            normalize_factor=1.0
            if self.normalize:
                nb_edge1=(self.A[g1][0:n*n] != torch.zeros(n*n,device=self.device)).int().sum()
                nb_edge2=(self.A[g2][0:m*m] != torch.zeros(m*m,device=self.device)).int().sum()
                normalize_factor=nodeInsDel*(n+m)+edgeInsDel*(nb_edge1+nb_edge2)
                
            ged[k]=.5*(v.t()@C@v)/normalize_factor    
        return ged
    
    def from_weighs_to_costs(self):
        
        #cn=torch.exp(self.node_weighs)
        #ce=torch.exp(self.edge_weighs)
        cn=self.node_weighs*self.node_weighs
        ce=self.edge_weighs*self.edge_weighs
        total_cost=cn.sum()+ce.sum()
        cn=cn/total_cost
        ce=ce/total_cost
        edgeInsDel=ce[-1]

        node_costs=torch.zeros((self.nb_labels,self.nb_labels),device=self.device)
        upper_part=torch.triu_indices(node_costs.shape[0],node_costs.shape[1],offset=1,device=self.device)        
        node_costs[upper_part[0],upper_part[1]]=cn[0:-1]
        node_costs=node_costs+node_costs.T

        if self.nb_edge_labels>1:
            edge_costs=torch.zeros((self.nb_edge_labels,self.nb_edge_labels),device=self.device)
            upper_part=torch.triu_indices(edge_costs.shape[0],edge_costs.shape[1],offset=1,device=self.device)        
            edge_costs[upper_part[0],upper_part[1]]=ce[0:-1]
            edge_costs=edge_costs+edge_costs.T
        else:
            edge_costs=torch.zeros(0,device=self.device)
        
        return node_costs,cn[-1],edge_costs,edgeInsDel
    
    def build_node_dictionnary(self,GraphList):
        #extraction de tous les labels d'atomes
        node_labels=[]
        for G in Gs:
            for v in nx.nodes(G):
                if not G.nodes[v][self.node_label][0] in node_labels:
                    node_labels.append(G.nodes[v][self.node_label][0])
        node_labels.sort()
        #extraction d'un dictionnaire permettant de numéroter chaque label par un numéro.
        dict={}
        k=0
        for label in node_labels:
            dict[label]=k
            k=k+1
        print(node_labels)
        print(dict,len(dict))
    
        return dict,max(max([[int(G[e[0]][e[1]]['bond_type']) for e in G.edges()] for G in GraphList]))
    
    def from_networkx_to_tensor(self,G,dict):    
        A=torch.tensor(nx.to_scipy_sparse_matrix(G,dtype=int,weight='bond_type').todense(),dtype=torch.int)        
        lab=[dict[G.nodes[v][self.node_label][0]] for v in nx.nodes(G)]
   
        return (A.view(1,A.shape[0]*A.shape[1]),torch.tensor(lab))

    def construct_cost_matrix(self,g1,g2,node_costs,edge_costs,nodeInsDel,edgeInsDel):
        n = self.card[g1].item()
        m = self.card[g2].item()
        
        A1=torch.zeros((n+1,n+1),dtype=torch.int,device=self.device)
        A1[0:n,0:n]=self.A[g1][0:n*n].view(n,n)
        A2=torch.zeros((m+1,m+1),dtype=torch.int,device=self.device)
        A2[0:m,0:m]=self.A[g2][0:m*m].view(m,m)
        
        
         # costs: 0 node subs, 1 nodeIns/Del, 2 : edgeSubs, 3 edgeIns/Del
        
        #C=cost[3]*torch.cat([torch.cat([C12[l][k] for k in range(n+1)],1) for l in range(n+1)])
        #Pas bien sur mais cela semble fonctionner.
        C=edgeInsDel*self.matrix_edgeInsDel(A1,A2)
        if self.nb_edge_labels>1:
            for k in range(self.nb_edge_labels):
                for l in range(self.nb_edge_labels):
                    if k != l:
#                    C.add_(self.matrix_edgeSubst(A1,A2,k+1,l+1),alpha=edge_costs[k][l])
                        C=C+edge_costs[k][l]*self.matrix_edgeSubst(A1,A2,k+1,l+1)
        #C=cost[3]*torch.tensor(np.array([ [  k!=l and A1[k//(m+1),l//(m+1)]^A2[k%(m+1),l%(m+1)] for k in range((n+1)*(m+1))] for l in range((n+1)*(m+1))]),device=self.device)        

        l1=self.labels[g1][0:n]
        l2=self.labels[g2][0:m]
        D=torch.zeros((n+1)*(m+1),device=self.device)
        D[n*(m+1):]=nodeInsDel
        D[n*(m+1)+m]=0
        D[[i*(m+1)+m for i in range(n)]]=nodeInsDel
        D[[k for k in range(n*(m+1)) if k%(m+1) != m]]=torch.tensor([node_costs[l1[k//(m+1)],l2[k%(m+1)]] for k in range(n*(m+1)) if k%(m+1) != m],device=self.device )
        mask = torch.diag(torch.ones_like(D))
        C=mask*torch.diag(D) + (1. - mask)*C
        
        #C[range(len(C)),range(len(C))]=D
      
        return C
    def matrix_edgeInsDel(self,A1,A2):
        Abin1=(A1!=torch.zeros((A1.shape[0],A1.shape[1]),device=self.device))
        Abin2=(A2!=torch.zeros((A2.shape[0],A2.shape[1]),device=self.device))
        C1=torch.einsum('ij,kl->ijkl',torch.logical_not(Abin1),Abin2)
        C2=torch.einsum('ij,kl->ijkl',Abin1,torch.logical_not(Abin2))
        C12=torch.logical_or(C1,C2).int()
    
        return torch.cat(torch.unbind(torch.cat(torch.unbind(C12,1),1),0),1)

    def matrix_edgeSubst(self,A1,A2,lab1,lab2):
        Abin1=(A1==lab1*torch.ones((A1.shape[0],A1.shape[1]),device=self.device)).int()
        Abin2=(A2==lab2*torch.ones((A2.shape[0],A2.shape[1]),device=self.device)).int()
        C=torch.einsum('ij,kl->ijkl',Abin1,Abin2)
        
        return torch.cat(torch.unbind(torch.cat(torch.unbind(C,1),1),0),1)
    
    def similarity_from_cost(self,C):
        N=C.shape[0]
             
        #return (torch.norm(C,p='fro')*torch.eye(N,device=self.device) -C)
        return (C.max()*torch.eye(N,device=self.device) -C)
    
    def mapping_from_cost(self,C,n,m):
        c=torch.diag(C)
        D=C-torch.eye(C.shape[0],device=self.device)*c
        x0=svd.eps_assigment_from_mapping(torch.exp(-c.view(n+1,m+1)),10).view((n+1)*(m+1),1) 
        # TODO : à améliorer : initialisation du FW
        #c = rings()
        #x0=svd.eps_assigment_from_mapping(torch.exp(-c.view(n+1,m+1)),10).view((n+1)*(m+1),1) 
        
        
        return svd.franck_wolfe(x0,D,c,5,15,n,m)
    
    def mapping_from_similarity(self,C,n,m):
        M=self.similarity_from_cost(C)
        first_ev=compute_major_axis(M)
        #first_ev=self.iterated_power(M,inv=True)
        if(first_ev.sum() <0):
            first_ev=-first_ev
        S=torch.exp(first_ev.view(n+1,m+1)) # enforce the difference, accelerate the convergence. 
        S=self.eps_assigment_from_mapping(S)
        return  S
        
    def eps_assigment_from_mapping(self,S):
        ones_n = torch.ones(S.shape[0],device=S.device)
        ones_m = torch.ones(S.shape[1],device=S.device)
    
        Sk = S
        for i in range(20):
            D=torch.diag(1.0/(Sk@ones_m))
            D[D.shape[0]-1,D.shape[1]-1]=1.0
            Sk1 = D@Sk
            D=torch.diag(1.0/(ones_n@Sk1))
            D[D.shape[0]-1,D.shape[1]-1]=1.0
            Sk = Sk1@D
        
        return Sk

print('Gs=',len(Gs))
model = Net(Gs,normalize=True,node_label='extended_label')

params = list(model.parameters())
print(len(params))
print(params[0])
#print(model(input))
print(max([G.order() for G in Gs]),len(Gs))
print('toto')

Gs= 68
['C_1C', 'C_1C1C1N', 'C_1C1C2C', 'C_1C1N', 'C_1C1N2C', 'C_1C1O', 'C_1C1O2C', 'C_1C2C', 'C_1C3C', 'C_1N', 'C_1O', 'C_2C', 'C_2C2C', 'C_3C', 'N_1C', 'N_1C1C', 'N_1C1C1C', 'O_1C', 'O_1C1C']
{'C_1C': 0, 'C_1C1C1N': 1, 'C_1C1C2C': 2, 'C_1C1N': 3, 'C_1C1N2C': 4, 'C_1C1O': 5, 'C_1C1O2C': 6, 'C_1C2C': 7, 'C_1C3C': 8, 'C_1N': 9, 'C_1O': 10, 'C_2C': 11, 'C_2C2C': 12, 'C_3C': 13, 'N_1C': 14, 'N_1C1C': 15, 'N_1C1C1C': 16, 'O_1C': 17, 'O_1C1C': 18} 19
3
torch.Size([68, 729])
adjacency matrices tensor([[0, 1, 0,  ..., 0, 0, 0],
        [0, 1, 0,  ..., 0, 0, 0],
        [0, 1, 0,  ..., 0, 0, 0],
        ...,
        [0, 1, 0,  ..., 0, 0, 0],
        [0, 1, 0,  ..., 0, 0, 0],
        [0, 1, 0,  ..., 0, 0, 0]], device='cuda:0', dtype=torch.int32)
node labels tensor([[15,  4,  7,  ...,  0,  0,  0],
        [15,  4,  7,  ...,  0,  0,  0],
        [15,  4,  7,  ...,  0,  0,  0],
        ...,
        [16,  4,  7,  ...,  0,  0,  0],
        [16,  4,  7,  ...,  0,  0,  0],
        [16,  4,  7,  ...,  

In [5]:
import itertools
nb=len(Gs)
class1=torch.tensor([k for k in range(len(y)) if y[k]==1])
class2=torch.tensor([k for k in range(len(y)) if y[k]==0])

train_size=15

if train_size % 2 == 0:
    nb_class1=int(train_size/2)
    nb_class2=int(train_size/2)
else:
    nb_class1=int(train_size/2)+1
    nb_class2=int(train_size/2)
    
print((torch.abs(10000*torch.randn(nb_class1)).int()%class1.size()[0]).long())
random_class1=class1[(torch.abs(10000*torch.randn(nb_class1)).int()%class1.size()[0]).long()]
random_class2=class2[(torch.abs(10000*torch.randn(nb_class2)).int()%class2.size()[0]).long()]
train_graphs=torch.cat((random_class1,random_class2),0)
print(train_graphs)

couples=torch.triu_indices(train_size,train_size,offset=1)

#combinations=itertools.combinations(range(nb),2)

nb_elt=int(train_size*(train_size-1)/2)
data=torch.empty((nb_elt,2),dtype=torch.int)
yt=torch.ones(nb_elt)

data[0:nb_elt,0]=train_graphs[couples[0]]
data[0:nb_elt,1]=train_graphs[couples[1]]
print(nb_elt)
#couples=[]
for k in range(nb_elt):
    if (y[data[k][0]]!=y[data[k][1]]):
        yt[k]=-1.0        

print('data=',data)

#print(couples[1:2])

torch.cuda.empty_cache()
if torch.cuda.is_available():
    device = torch.device("cuda:0")          # a CUDA device object    


tensor([ 2, 12, 27, 18,  9, 27,  8, 14])
tensor([20,  4, 37, 14, 44, 46, 33, 37, 42, 40, 64, 13,  6,  5,  9])
105
data= tensor([[20,  4],
        [20, 37],
        [20, 14],
        [20, 44],
        [20, 46],
        [20, 33],
        [20, 37],
        [20, 42],
        [20, 40],
        [20, 64],
        [20, 13],
        [20,  6],
        [20,  5],
        [20,  9],
        [ 4, 37],
        [ 4, 14],
        [ 4, 44],
        [ 4, 46],
        [ 4, 33],
        [ 4, 37],
        [ 4, 42],
        [ 4, 40],
        [ 4, 64],
        [ 4, 13],
        [ 4,  6],
        [ 4,  5],
        [ 4,  9],
        [37, 14],
        [37, 44],
        [37, 46],
        [37, 33],
        [37, 37],
        [37, 42],
        [37, 40],
        [37, 64],
        [37, 13],
        [37,  6],
        [37,  5],
        [37,  9],
        [14, 44],
        [14, 46],
        [14, 33],
        [14, 37],
        [14, 42],
        [14, 40],
        [14, 64],
        [14, 13],
        [14,  6],
        [14,  5]

In [6]:
#if torch.cuda.device_count() > 1:
#  print("Let's use", torch.cuda.device_count(), "GPUs!")
  # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
#  model = nn.DataParallel(model)
from triangular_losses import TriangularConstraint as triangular_constraint
model.to(device)
def classification(model,data,yt,nb_iter):

    criterion = torch.nn.HingeEmbeddingLoss(margin=1.0)
    criterionTri=triangular_constraint()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    print(device)

#torch.cat((same_class[0:20],diff_class[0:20]),0).to(device)
    input=data.to(device)
    target=yt.to(device) 
#torch.ones(40,device=device)
#target[20:]=-1.0
#target=(yt[0:20]).to(device)
    InsDel=np.empty((nb_iter,2))
    node_costs,nodeInsDel,edge_costs,edgeInsDel=model.from_weighs_to_costs()
    nodeSub=np.empty((nb_iter,int(node_costs.shape[0]*(node_costs.shape[0]-1)/2)))
    edgeSub=np.empty((nb_iter,int(edge_costs.shape[0]*(edge_costs.shape[0]-1)/2)))
    loss_plt=np.empty(nb_iter)
    for t in range(nb_iter):    
        # Forward pass: Compute predicted y by passing data to the model    


        y_pred = model(input).to(device)
    
    
        # Compute and print loss
        loss = criterion(y_pred, target).to(device)    
        node_costs,nodeInsDel,edge_costs,edgeInsDel=model.from_weighs_to_costs()
        triangularInq=criterionTri(node_costs,nodeInsDel,edge_costs,edgeInsDel)
        loss=loss*(1+triangularInq)
        loss.to(device)
        InsDel[t][0]=nodeInsDel.item()
        InsDel[t][1]=edgeInsDel.item()
        loss_plt[t]=loss.item()
        print(t, loss.item())  
        k=0
        for p in range(node_costs.shape[0]):
            for q in range(p+1,node_costs.shape[0]):
                nodeSub[t][k]=node_costs[p][q]
                k=k+1
        k=0
        for p in range(edge_costs.shape[0]):
            for q in range(p+1,edge_costs.shape[0]):
                edgeSub[t][k]=edge_costs[p][q]
                k=k+1
        if t % 100 == 99 or t==0:                         
            print('Distances: ',y_pred)
            print('Loss Triangular:',triangularInq.item())
            print('node_costs :')
            print(node_costs)
            print('nodeInsDel:',nodeInsDel.item())
            print('edge_costs :')
            print(edge_costs)
            print('edgeInsDel:',edgeInsDel.item())
        
        
        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return InsDel,nodeSub,edgeSub,loss_plt

nb_iter=200
InsDel, nodeSub,edgeSub,loss_plt=classification(model,data,yt,nb_iter)
plt.figure(0)
plt.plot(InsDel[0:nb_iter,0],label="node")
plt.plot(InsDel[0:nb_iter,1],label="edge")
plt.title('Node/Edge insertion/deletion costs')
plt.legend()
plt.figure(1)
for k in range(nodeSub.shape[1]):
    plt.plot(nodeSub[0:nb_iter,k])
plt.title('Node Substitutions costs')
plt.figure(2)
for k in range(edgeSub.shape[1]):
    plt.plot(edgeSub[0:nb_iter,k])
plt.title('Edge Substitutions costs')
plt.figure(3)
plt.plot(loss_plt)
plt.title('Evolution of the loss')
plt.show()
plt.close()

cuda:0
0 0.4992794096469879
Distances:  tensor([0.2360, 0.2595, 0.2360, 0.2520, 0.2595, 0.2521, 0.2595, 0.3059, 0.3007,
        0.3604, 0.2181, 0.2360, 0.2520, 0.2361, 0.2763, 0.2527, 0.2689, 0.2764,
        0.2689, 0.2763, 0.3649, 0.3625, 0.3672, 0.2345, 0.2527, 0.2688, 0.2529,
        0.2282, 0.2441, 0.2517, 0.2441, 0.2515, 0.2984, 0.2930, 0.3034, 0.2108,
        0.2282, 0.2441, 0.2285, 0.2689, 0.2764, 0.2689, 0.2763, 0.3649, 0.3625,
        0.3672, 0.2345, 0.2527, 0.2688, 0.2529, 0.2595, 0.2519, 0.2593, 0.3060,
        0.3007, 0.3602, 0.2181, 0.2359, 0.2519, 0.2362, 0.2443, 0.2517, 0.2984,
        0.2931, 0.3035, 0.2108, 0.2284, 0.2443, 0.2285, 0.2593, 0.3060, 0.3007,
        0.3602, 0.2182, 0.2359, 0.2519, 0.2361, 0.2984, 0.2930, 0.3034, 0.2108,
        0.2282, 0.2441, 0.2285, 0.2468, 0.2575, 0.1687, 0.1845, 0.1991, 0.1845,
        0.2631, 0.1734, 0.1895, 0.2043, 0.1896, 0.1643, 0.1797, 0.1941, 0.1798,
        0.2718, 0.2879, 0.2719, 0.2688, 0.2529, 0.2361], device='cuda:0',
      

KeyboardInterrupt: 

In [ ]:
A=torch.tensor(nx.to_scipy_sparse_matrix(Gs[0],dtype=int,weight='bond_type').todense(),dtype=torch.int) 
print(A)

In [ ]:
plt.figure(0)
plt.plot(InsDel[0:500,0],label="node")
plt.plot(InsDel[0:500,1],label="edge")
plt.title('Node/Edge insertion/deletion costs')
plt.legend()
plt.figure(1)
for k in range(nodeSub.shape[1]):
    plt.plot(nodeSub[0:500,k])
plt.title('node Substitution costs')
plt.figure(2)
for k in range(edgeSub.shape[1]):
    plt.plot(edgeSub[0:500,k])
plt.title('edge Substitution costs')
plt.show()
plt.close()